In [1]:
-- Data Duration
SELECT 
    (SELECT MAX(date) FROM Covid_Deaths) AS Max_Date_Deaths_Table,
	(SELECT MIN(date) FROM Covid_Deaths) AS Min_Date_Deaths_Table,
    (SELECT MAX(date) FROM Covid_Vaccinations) AS Max_Date_Vaccination_Table,
    (SELECT MIN(date) FROM Covid_Vaccinations) AS Min_Date_Vaccination_Table
--Findings: dataset contains data Fom January 1, 2020 to April 30, 2021,


(1 row affected)

Total execution time: 00:00:00.174

Max_Date_Deaths_Table,Min_Date_Deaths_Table,Max_Date_Vaccination_Table,Min_Date_Vaccination_Table
2021-04-30 00:00:00.000,2020-01-01 00:00:00.000,2021-04-30 00:00:00.000,2020-01-01 00:00:00.000


In [2]:
--1. Total cases, deaths, and vaccinations over time (Monthly)(Globally)
SELECT 
       FORMAT(D.date, 'yyyy-MM') AS Month,
       SUM(TRY_CAST(D.total_cases AS FLOAT)) AS Total_Cases,
	   SUM(TRY_CAST(D.total_deaths AS FLOAT)) AS Total_Deaths,
	   SUM(TRY_CAST(V.total_vaccinations AS FLOAT)) AS Total_Vaccinations
FROM dbo.Covid_Deaths D
JOIN dbo.Covid_Vaccinations V
     ON D.iso_code=V.iso_code
	AND D.date=V.date
WHERE D.location = 'World'  -- Focuses only on global data
GROUP BY FORMAT(D.date, 'yyyy-MM')
ORDER BY Month
--Findings: 
--1. Cases & Deaths Surged in Late 2020 – Rapid increase from October to December 2020 (Cases: 1.2B → 2.3B, Deaths: 34M → 51M).
--2. Vaccinations Began in December 2020 – Slow start but reached 26B+ doses by April 2021.
--3. Cases Still Peaked in April 2021 – Despite vaccinations, total cases hit 4.19B, showing infections were still spreading.
--4. Cases did not drop immediately, peaking in April 2021 despite mass vaccinations.
--5.Deaths slowed down after vaccinations increased, suggesting some impact but not an instant decline.


Warning: Null value is eliminated by an aggregate or other SET operation.

(16 rows affected)

Total execution time: 00:00:12.997

Month,Total_Cases,Total_Deaths,Total_Vaccinations
2020-01,38538,889,NULL
2020-02,1672032,46910,NULL
2020-03,9057318,399150,NULL
2020-04,63570406,4361073,NULL
2020-05,145211078,9684444,NULL
2020-06,246660148,13297463,NULL
2020-07,431779916,18283969,NULL
2020-08,671687811,23814819,NULL
2020-09,894819761,28127317,NULL
2020-10,1229233464,34293432,NULL


In [3]:
--2. Top 10 Countries with Highest Cases, Deaths, and Vaccinations (Highest to Lowest)
SELECT TOP 10 D.location,
       MAX(TRY_CAST(D.total_cases AS FLOAT)) AS Max_Cases,
	   MAX(TRY_CAST(D.total_deaths AS FLOAT)) AS Max_Deaths,
	   MAX(TRY_CAST(V.total_vaccinations AS FLOAT)) AS Max_Vaccinations
FROM dbo.Covid_Deaths D
JOIN dbo.Covid_Vaccinations V
     ON D.iso_code = V.iso_code
    AND D.date = V.date
WHERE D.continent IS NOT NULL -- Excludes continents & world-level data
GROUP BY D.location
ORDER BY Max_Cases DESC -- Sorting by highest cases
--Findings: 
--1. Countries with high cases also had high vaccinations, showing mass immunization efforts.
--2. Brazil had the highest deaths among non-U.S. countries, indicating severe outbreaks.
--3. European countries had high vaccinations despite fewer cases than the U.S. and India.


Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:17.297

location,Max_Cases,Max_Deaths,Max_Vaccinations
United States,32346971,576232,240159677
India,19164969,211853,151998107
Brazil,14659011,403781,42698862
France,5677835,104675,21518866
Turkey,4820591,40131,22816891
Russia,4750755,108290,19834392
United Kingdom,4432246,127775,48748962
Italy,4022653,120807,19949067
Spain,3524077,78216,16364595
Germany,3405365,83097,28774580


In [4]:
--3. Top 10 countries with the highest Mortality Rate
--NULLIF(..., 0) prevents division by zero by returning NULL if a country has 0 total cases.
SELECT TOP 10 location AS Country,
       SUM(TRY_CAST(total_deaths AS FLOAT)) 
	   / NULLIF(SUM(TRY_CAST(total_cases AS FLOAT)), 0) * 100 AS Mortality_Rate
FROM dbo.Covid_Deaths
WHERE continent IS NOT NULL  -- Excludes aggregated regions like "World"
GROUP BY location
ORDER BY Mortality_Rate DESC;
-- Findings:
--1. Mortality Rate represents the percentage of people who died after getting infected with COVID-19 in a given country.
--2. Yemen (25.94%) had the highest mortality rate, meaning about 26 out of every 100 infected people died—likely due to poor healthcare and low testing.
--3. Mexico (9.35%) had the second-highest rate, possibly due to underreporting of cases.
--4. Sudan (6.29%) and Syria (6.20%) also showed high mortality, likely due to war-affected healthcare systems.


Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:00.199

Country,Mortality_Rate
Yemen,25.944440560255423
Mexico,9.354697388207853
Ecuador,6.306005803439198
Sudan,6.29533819898756
Syria,6.204207741851854
Egypt,5.605203468124468
Liberia,5.114233690825743
Bolivia,5.0192932922432405
China,4.922602656916756
Chad,4.915356816947238


In [5]:
-- 4. Country with the highest daily spike cases
SELECT Top 1 location AS Country, 
       date, 
       MAX(TRY_CAST(new_cases AS FLOAT)) AS Highest_Daily_Cases
FROM dbo.Covid_Deaths
WHERE continent IS NOT NULL  -- Excludes aggregated regions like "World"
GROUP BY location, date
ORDER BY Highest_Daily_Cases DESC;
--Findings: 
--1. India recorded the highest daily spike in COVID-19 cases on April 30, 2021, with 401,993 new cases.
--2. This was during India's second wave, driven by the Delta variant and healthcare shortages.
--3. It marked one of the worst single-day surges globally.


Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:07.537

Country,date,Highest_Daily_Cases
India,2021-04-30 00:00:00.000,401993


In [6]:
--5. Deadliest Month (Highest Monthly Deaths)
SELECT FORMAT(D.date, 'yyyy-MM') AS Month,
       SUM(TRY_CAST(D.new_deaths AS FLOAT)) AS Monthly_Deaths
FROM dbo.Covid_Deaths D
WHERE D.new_deaths IS NOT NULL
GROUP BY FORMAT(D.date, 'yyyy-MM')
ORDER BY Monthly_Deaths DESC;
--Findings:
--1. January 2021 had the highest number of deaths.
--2. December 2020 & April 2021 were also critical months.
--3. Early pandemic months (Jan–Mar 2020) had relatively fewer deaths due to limited spread initially.
--4. Vaccination rollout began in early 2021, but high deaths still occurred in early months before immunity developed.



(16 rows affected)

Total execution time: 00:00:00.597

Month,Monthly_Deaths
2021-01,1333535
2021-04,1164351
2020-12,1163213
2021-02,967050
2021-03,919198
2020-11,917349
2020-04,655044
2020-10,572083
2020-08,530611
2020-07,504983


In [7]:
--6. Top 2 Countries by percentage of population vaccinated
SELECT TOP 2 
    D.location AS Country,
    MAX(TRY_CAST(V.people_fully_vaccinated AS FLOAT)) 
    / NULLIF(MAX(TRY_CAST(D.population AS FLOAT)), 0) * 100 AS Vaccination_Percentage
FROM dbo.Covid_Deaths D
JOIN dbo.Covid_Vaccinations V 
    ON D.iso_code = V.iso_code 
    AND D.date = V.date
WHERE D.continent IS NOT NULL -- Exclude aggregated regions like 'World'
GROUP BY D.location
ORDER BY Vaccination_Percentage DESC;
--Findings: Gibraltar leads globally with 97.47% of its population fully vaccinated. Seychelles at 60.16%.


Warning: Null value is eliminated by an aggregate or other SET operation.

(2 rows affected)

Total execution time: 00:00:00.178

Country,Vaccination_Percentage
Gibraltar,97.4681665726752
Seychelles,60.158633312995725


In [8]:
--7. Which continents handled the pandemic better?
--Mortality Rate: Lower is better.
--Test Positivity Rate: Lower means better testing & control.
--Vaccination Coverage: Higher means better prevention.
---Sorted by Mortality Rate (ASC) since lower death rates indicate better handling.
SELECT 
    D.continent AS Continent,
    SUM(TRY_CAST(D.total_deaths AS FLOAT)) / NULLIF(SUM(TRY_CAST(D.total_cases AS FLOAT)), 0) * 100 AS Mortality_Rate,
    SUM(TRY_CAST(D.total_cases AS FLOAT)) / NULLIF(SUM(TRY_CAST(V.total_tests AS FLOAT)), 0) * 100 AS Test_Positivity_Rate,
    MAX(TRY_CAST(V.people_fully_vaccinated AS FLOAT)) / NULLIF(MAX(TRY_CAST(D.population AS FLOAT)), 0) * 100 AS Vaccination_Coverage
FROM dbo.Covid_Deaths D
JOIN dbo.Covid_Vaccinations V 
    ON D.iso_code = V.iso_code 
    AND D.date = V.date
WHERE D.continent IS NOT NULL  -- Exclude 'World' and other aggregate regions
GROUP BY D.continent
ORDER BY Mortality_Rate ASC;  -- Lower mortality = better handling
--Findings: 
--1. Oceania handled the pandemic the best (low cases, low mortality).
--2. South America struggled the most (high mortality, high case rates).
--3. Asia & Europe managed relatively well with control measures and healthcare support.


Warning: Null value is eliminated by an aggregate or other SET operation.

(6 rows affected)

Total execution time: 00:00:00.214

Continent,Mortality_Rate,Test_Positivity_Rate,Vaccination_Coverage
Asia,1.6608709454864103,6.966958775639201,1.8495598753307327
Africa,2.5231021727475595,15.757941877721251,2.0667607139428292
Oceania,2.6027177225671996,0.41063571851420516,0.2353893337776753
North America,2.6033267623612244,8.136270706271423,30.636406965047623
Europe,2.7164670042737837,8.542236674050883,9.958494381655985
South America,2.833960681294879,60.08913086247812,6.374382608487587
